In [1]:
from datasets import load_dataset
from transformers import BertTokenizerFast, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
from transformers import DataCollatorWithPadding
import pickle
import os
from sklearn.metrics import classification_report, accuracy_score, f1_score,ConfusionMatrixDisplay, confusion_matrix
import pickle
os.environ["WANDB_DISABLED"] = "true"

c:\Users\zhanr24\.conda\envs\diag\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch

In [3]:
torch.cuda.is_available()

True

In [4]:
model_name = "bert-base-uncased"
max_length=128

In [5]:
le = pickle.load(open("Data/label_encoder.pkl", "rb"))

c:\Users\zhanr24\.conda\envs\diag\lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.0.2 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [6]:
tokenizer = BertTokenizerFast.from_pretrained(model_name, do_lower_case=True)

In [7]:
target_names=le.transform(le.classes_)

In [8]:
def preprocess_function(examples):
    #print(examples['TEXT'])
    return tokenizer(examples["text"], padding=True, truncation=True)
    #return toknizer(examples["text"],padding='max_length',truncation=True, max_length=max_length)

In [9]:
def get_prediction(text):
    # prepare our text into tokenized sequence
    inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt").to("cuda")
    
    # perform inference to our model
    outputs = model(**inputs)
    #print(outputs)
    # get output probabilities by doing softmax
    probs = outputs[0].softmax(1)
    #print(outputs[0].softmax(1).argmax())
    # executing argmax function to get the candidate label
    return probs.argmax().item()

In [10]:
best_params = {"batch_size":8, 
              "lr": 5e-5,
              "epochs": 40}

In [11]:
train_dataset = load_dataset('csv', data_files={'train': 'Data/BERT_data_for_training'+'.csv'})
#print(train_dataset["train"][0])

tokenized_dataset_train =train_dataset.map(preprocess_function)

                                                
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
#dataset has been loaded correctly. 


In [12]:
model=BertForSequenceClassification.from_pretrained(model_name, num_labels=13)
#device="cpu"
device="cuda"
model.to(device)

#initializing training arg ith bet params
training_args = TrainingArguments(
num_train_epochs=best_params["epochs"],
learning_rate=best_params["lr"],
per_device_train_batch_size=best_params["batch_size"],  # batch size per device during training
weight_decay=0.01,               # strength of weight decay
#oad_best_model_at_end=True,
logging_steps=100,
#valuation_strategy="steps",
output_dir="Models/BERT_13classes_final_output"
)


trainer = Trainer( model=model,
              args=training_args,
              train_dataset=tokenized_dataset_train['train'],
              tokenizer=tokenizer,
              data_collator=data_collator
             )


trainer.train()

trainer.save_model("Final_Model/BERT_13classes_final")


A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

{'loss': 1.5295, 'grad_norm': 12.741456985473633, 'learning_rate': 4.8798076923076926e-05, 'epoch': 0.96}


  5%|▍         | 203/4160 [00:15<04:09, 15.86it/s]

{'loss': 0.6369, 'grad_norm': 0.6792770624160767, 'learning_rate': 4.7596153846153844e-05, 'epoch': 1.92}


  7%|▋         | 303/4160 [00:22<04:10, 15.40it/s]

...

{'loss': 0.0235, 'grad_norm': 0.012210567481815815, 'learning_rate': 1.9230769230769234e-06, 'epoch': 38.46}


 99%|█████████▊| 4101/4160 [10:22<00:03, 15.37it/s]

{'loss': 0.0151, 'grad_norm': 0.009663000702857971, 'learning_rate': 7.211538461538462e-07, 'epoch': 39.42}


100%|██████████| 4160/4160 [11:08<00:00,  6.23it/s]


{'train_runtime': 668.2208, 'train_samples_per_second': 49.505, 'train_steps_per_second': 6.225, 'train_loss': 0.09467245889111207, 'epoch': 40.0}


## Eval

In [23]:
import torch

In [8]:
model_path = 'Models\Final_model\BERT_13classes_final'

In [10]:
!git lfs pull

Not in a Git repository.


In [11]:
model=BertForSequenceClassification.from_pretrained(model_path, num_labels=13)
tokenizer = BertTokenizerFast.from_pretrained(model_path, do_lower_case=True)

OSError: You seem to have cloned a repository without having git-lfs installed. Please install git-lfs and run `git lfs install` followed by `git lfs pull` in the folder you cloned.

In [34]:
def get_new_prediction(text):
    # prepare our text into tokenized sequence
    inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt").to("cuda")
   # print(inputs)
    # perform inference to our model
    outputs = model(**inputs)
    #print(outputs)
    # get output probabilities by doing softmax
    probs = outputs[0].softmax(1).argmax()
    #print(probs.item())
    #print(outputs[0].softmax(1).argmax())
    # executing argmax function to get the candidate label
    return probs.item()
    #return probs.argmax().item()



labels_dict = {'y': 'yes-answers',
               'y-d': 'yes-answers-explanations',
               'n': 'no-answers',
               'n-d': 'no-answers-explanations',
               'sno': 'statements-non-opinion',
               'so': 'statements-opinion',
               'ack': 'acknowledgments',
               'dno': "other answers",
               'query': 'interviewee-initiated questions',
               'ft': 'thanking',
               'fa': 'apologies', 
               'fe': 'explanations',
               'fp': 'conventional'
              }

In [35]:
text = "i think this works well"


In [36]:
print("The label is: ", labels_dict[le.inverse_transform([get_new_prediction(text)])[0]])

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument index in method wrapper__index_select)

In [16]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element